In [14]:
import pandas as pd
import os

In [15]:
experiment_type_A = 'reward-network-iii-8A-v1'
experiment_type_B = 'reward-network-iii-8B-v1'

In [47]:
moves_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type_B,'moves.csv'))
rel_trial_types = ['try_yourself']

moves_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)]

opt_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)].groupby(['session_id', 'trial_idx'])['optimal'].sum().reset_index()
opt_df['all_optimal'] = opt_df['optimal'] == 10
opt_df['all_optimal_discovered'] = opt_df.groupby('session_id')['all_optimal'].transform('cummax')

opt_df['not_discovered'] = ~opt_df['all_optimal_discovered']

opt_player_df = opt_df.groupby('session_id')['not_discovered'].sum().reset_index()
opt_player_df.shape[0] / opt_player_df['not_discovered'].sum()

0.5263157894736842

In [59]:
moves_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type_A,'moves.csv'))
rel_trial_types = ['individual', 'demonstration']

moves_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)]

opt_df = moves_df[moves_df['trial_type'].isin(rel_trial_types)].groupby(['session_id', 'trial_idx'])['optimal'].sum().reset_index()
opt_df['all_optimal'] = opt_df['optimal'] == 10

(opt_df['all_optimal'].sum() + 2) / (opt_df.shape[0] + 100)

0.010526315789473684